In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_ohc_bin.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_ohc_bin.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [3]:
X_submit = pd.read_csv('data/X_submit_ohc_bin.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')


In [4]:
X_train.shape

(179691, 81)

In [5]:
X_test.shape

(59897, 81)

In [6]:
X_submit.shape

(60000, 81)

In [7]:
randomForest = RandomForestRegressor(n_estimators=1200, n_jobs= 5, oob_score=True, min_samples_leaf=2, max_features= 0.5)

In [8]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [9]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [10]:
def print_score(r,train_set,test_set,train_target,test_target,F):
    res = [F(r.predict(train_set), train_target),
           F(r.predict(test_set), test_target),
           r.score(train_set, train_target), r.score(test_set, test_target)]
    if hasattr(r, 'oob_score_'): res.append(r.oob_score_)
    print(res)

In [11]:
%time randomForest.fit(X_train, y_train_log)

CPU times: user 34min 34s, sys: 6.55 s, total: 34min 41s
Wall time: 7min 38s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=5,
                      oob_score=True, random_state=None, verbose=0,
                      warm_start=False)

In [10]:
%time print_score(randomForest,X_train,X_test,y_train_log,y_test_log,RMSE)

[0.16630426646531893, 0.3289944642225686, 0.9605023769081238, 0.8466793128625428, 0.846168297369963]
CPU times: user 7min 26s, sys: 992 ms, total: 7min 27s
Wall time: 1min 31s


In [13]:
#Primer submit
serie_submit = pd.Series(data=np.exp(randomForest.predict(X_submit)),index=X_submit.index,name='target')

In [14]:
df_entrega = serie_submit.reset_index()
df_entrega.head()

,id,target
0,4941,5.310950e+06
1,51775,8.973424e+05
2,115253,2.441358e+06
3,299321,1.382366e+06
4,173570,5.832688e+05


In [15]:
df_entrega.to_csv('data/submit2.csv',index=False)

In [ ]:
#[0.17535284945308224, 0.3390403183772477, 0.9560873310026407, 0.8371730565360751, 0.8351937688714631]
#CPU times: user 1min 6s, sys: 220 ms, total: 1min 6s
#Wall time: 14.6 s